In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
# df = pd.read_csv(file_path + 'Oanda_Aud_Usd_M5_2021-2022.csv')
# df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2021-2022.csv')
# df = pd.read_csv(file_path + 'Oanda_Usd_Cad_M5_2021-2022.csv')
df = pd.read_csv(file_path + 'Oanda_Gbp_Chf_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-04-01 06:00:00   1.30152   1.30160  1.30133    1.30143   1.30174   
1 2021-04-01 06:05:00   1.30145   1.30146  1.30084    1.30087   1.30165   
2 2021-04-01 06:10:00   1.30085   1.30115  1.30062    1.30062   1.30106   
3 2021-04-01 06:15:00   1.30064   1.30092  1.30058    1.30072   1.30086   
4 2021-04-01 06:20:00   1.30074   1.30074  1.30006    1.30038   1.30090   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   1.30180  1.30155    1.30165   1.30163   1.30170  1.30144    1.30154   
1   1.30166  1.30102    1.30106   1.30155   1.30155  1.30094    1.30096   
2   1.30136  1.30080    1.30085   1.30096   1.30126  1.30071    1.30074   
3   1.30117  1.30081    1.30090   1.30075   1.30104  1.30070    1.30081   
4   1.30091  1.30026    1.30059   1.30082   1.30082  1.30016    1.30048   

   Volume  
0     509  
1     643  
2     475  
3     562  
4     799  
--------------------------

In [5]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def range_high_lows(df, i, n_histograms=6):
    green, red = False, False
    curr_color, curr_streak = None, 0
    high, low = -np.inf, np.inf

    while i >= 0:
        hist = df.loc[df.index[i], 'macdhist']
        curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
        new_color = 'green' if hist > 0 else 'red'

        if curr_color != new_color:
            curr_color = new_color
            curr_streak = 0

            if green and red:
                break

        curr_streak += 1

        if curr_streak >= n_histograms:
            high = max(high, curr_high)
            low = min(low, curr_low)

            if curr_color == 'red':
                red = True

            else:
                green = True

        i -= 1

    return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan

# def range_high_lows(df, i):
#     green, red = False, False
#     curr_color, curr_streak = None, 0
#     high, low = -np.inf, np.inf

#     while i >= 0:
#         hist = df.loc[df.index[i], 'macdhist']
#         curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
#         new_color = 'green' if hist > 0 else 'red'

#         if curr_color != new_color:
#             curr_color = new_color
#             curr_streak = 0

#             if green and red:
#                 break

#         curr_streak += 1

#         if curr_streak >= 3:
#             if curr_color == 'red':
#                 low = min(low, curr_low)
#                 red = True

#             else:
#                 high = max(high, curr_high)
#                 green = True

#         i -= 1

#     return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan


In [6]:
# Add technical indicators (for additional features)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=25).mean()

df['atr'] = atr(df)
df['rsi'] = rsi(df, periods=18)
df['rsi_sma'] = df['rsi'].rolling(36).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['macdhist'] = df['macd'] - df['macdsignal']
range_highs_lows_tups = [range_high_lows(df, idx) for idx in range(len(df))]
df['range_high'], df['range_low'] = [tup[0] for tup in range_highs_lows_tups], [tup[1] for tup in range_highs_lows_tups]
df['n_macd'], df['n_macdsignal'] = n_macd(df['macd'], df['macdsignal'])
df['slowk'], df['slowd'] = stoch(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])

df['sar'] = psar(df)

df['chop14'] = chop(df)
df['chop36'] = chop(df, lookback=36)

df['vo'] = vo(df['Volume'])
df['vo_long'] = vo(df['Volume'], short_lookback=18, long_lookback=36)

df['len_avg'], df['len_std'] = bar_lengths(abs(df['Mid_High'] - df['Mid_Low']))
df['sar_len_avg'], df['sar_len_std'] = sar_lengths(df['Mid_Open'], df['sar'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-04-01 10:25:00   1.30249   1.30286  1.30243    1.30281   1.30268   
1     2021-04-01 10:30:00   1.30279   1.30292  1.30215    1.30232   1.30296   
2     2021-04-01 10:35:00   1.30231   1.30238  1.30154    1.30183   1.30247   
3     2021-04-01 10:40:00   1.30183   1.30208  1.30136    1.30199   1.30201   
4     2021-04-01 10:45:00   1.30199   1.30307  1.30195    1.30277   1.30216   
...                   ...       ...       ...      ...        ...       ...   
74746 2022-04-01 05:35:00   1.21336   1.21375  1.21331    1.21370   1.21358   
74747 2022-04-01 05:40:00   1.21370   1.21372  1.21330    1.21332   1.21387   
74748 2022-04-01 05:45:00   1.21332   1.21354  1.21323    1.21340   1.21353   
74749 2022-04-01 05:50:00   1.21340   1.21352  1.21323    1.21331   1.21361   
74750 2022-04-01 05:55:00   1.21335   1.21339  1.21309    1.21314   1.21352   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  ... 

In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [8]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'JPY' else pips_to_risk * 100

    if second == 'USD':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'JPY' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, chop_cutoff, chop_num, lookback, use_adx):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    limit_order = None

    for i in range(lookback, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        # ema200_1, ema100_1, ema50_1, ema25_1, atr1, sar1, rsi1, rsi_sma1, mid_open1, mid_close1, mid_low1, mid_high1, vo1, adx1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'rsi_sma', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High', 'vo', 'adx']]
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_mid_open, curr_mid_low = \
            df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_Low']]
        chop_avg = df.loc[df.index[i - lookback:i], f'chop{chop_num}'].mean() 
        adx_signal = max(list(df.loc[df.index[i - lookback:i], 'adx'])) < 20 if use_adx else True 
        chop = df.loc[df.index[i - 1], f'chop{chop_num}']
        chop_signal = adx_signal if use_adx else chop_avg > chop_cutoff
        stop_signal = df.loc[df.index[i - 1], 'adx'] > 20 if use_adx else chop < chop_cutoff

        # vo = df.loc[df.index[i - 1], 'vo_long']

        # if limit_order is None and trade is None and chop_avg > chop_cutoff:
        if limit_order is None and trade is None and chop_signal:
            range_high, range_low = df.loc[df.index[i - 1], ['range_high', 'range_low']]
            
            if range_high > range_low:
                limit_order = {'buy': float(range_low), 'sell': float(range_high), 'range': abs(float(range_high) - float(range_low))}

        # if chop < chop_cutoff:
        #     limit_order = None

        if stop_signal:
            limit_order = None

        if trade is None and limit_order is not None:
            place_buy = curr_ask_low <= limit_order['buy']
            place_sell = curr_bid_high >= limit_order['sell']

            if place_buy:
                open_price = float(limit_order['buy'])
                stop_gain = open_price + limit_order['range'] / 2
                pips_gained = stop_gain - open_price
                curr_pips_to_risk = pips_gained / risk_reward_ratio
                pullback = open_price - curr_pips_to_risk

                stop_loss = round(pullback, 5)

                if stop_loss < open_price and stop_gain > open_price and spread <= curr_pips_to_risk * spread_cutoff:
                      n_units = get_n_units('buy', stop_loss, open_price, curr_bid_open, curr_mid_open, 'USD_CAD')

                      trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                      'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                      'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                      n_buys += 1

                      pips_risked.append(curr_pips_to_risk)
                      
                      limit_order = None

            elif place_sell:
                open_price = float(limit_order['sell'])
                stop_gain = open_price - limit_order['range'] / 2
                pips_gained = open_price - stop_gain
                curr_pips_to_risk = pips_gained / risk_reward_ratio
                pullback = open_price + curr_pips_to_risk

                stop_loss = round(pullback, 5)

                if stop_loss > open_price and stop_gain < open_price and spread <= curr_pips_to_risk * spread_cutoff:
                      n_units = get_n_units('sell', stop_loss, curr_ask_open, open_price, curr_mid_open, 'USD_CAD')

                      trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                              'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                      n_sells += 1

                      pips_risked.append(curr_pips_to_risk)

                      limit_order = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [9]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [0.5, 1.0, 1.5]
# risk_reward_ratio_vals = [1.5]
spread_cutoffs = [0.10, 0.15, 0.20]
# spread_cutoffs = [0.20]
chop_cutoff_vals = [0.50, 0.559, 0.618]
# chop_cutoff_vals = [0.50]
# chop_nums = [14, 36]
chop_nums = [14]
lookback_vals = [1, 3, 5, 6]
use_adx_vals = [True, False]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(chop_cutoff_vals) * len(chop_nums) * len(lookback_vals) * len(use_adx_vals)
all_combos = []


# use_half_sar, vo_cutoff, use_candles

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for chop_cutoff in chop_cutoff_vals:
            for chop_num in chop_nums:
                for lookback in lookback_vals:
                    for use_adx in use_adx_vals:
                        all_combos.append((risk_reward_ratio, spread_val, chop_cutoff, chop_num, lookback, use_adx))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_chop_cutoff_val = None
best_chop_num = None
best_lookback = None
best_use_adx_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, chop_cutoff, chop_num, lookback, use_adx in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, chop_cutoff, chop_num, lookback, use_adx)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    # print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread_val': spread_val, 'chop_cutoff': chop_cutoff, 'chop_num': chop_num, 'lookback': lookback, 'use_adx': use_adx})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_chop_cutoff_val = chop_cutoff
        best_chop_num = chop_num
        best_lookback = lookback
        best_use_adx_val = use_adx

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 216

-3454.320979999998
Num buys: 143
Num sells: 105
Num trades: 248
Num wins: 97
Num losses: 150
Win streak: 6
Loss streak: 8
Remaining runs: 215
Best reward so far: -3454.320979999998

-4471.951250000109
Num buys: 354
Num sells: 354
Num trades: 708
Num wins: 427
Num losses: 280
Win streak: 10
Loss streak: 7
Remaining runs: 214
Best reward so far: -3454.320979999998

-7894.227195000155
Num buys: 296
Num sells: 285
Num trades: 581
Num wins: 229
Num losses: 351
Win streak: 5
Loss streak: 23
Remaining runs: 213
Best reward so far: -3454.320979999998

-1038.3992400000177
Num buys: 22
Num sells: 12
Num trades: 34
Num wins: 7
Num losses: 27
Win streak: 2
Loss streak: 13
Remaining runs: 212
Best reward so far: -1038.3992400000177

-3085.2992250001025
Num buys: 315
Num sells: 282
Num trades: 597
Num wins: 367
Num losses: 229
Win streak: 9
Loss streak: 5
Remaining runs: 211
Best reward so far: -1038.3992400000177

-1897.1478200000226
Num buys: 43
Num sells: 27
Num trades: 70
Num wins

KeyboardInterrupt: 

In [10]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best chop cutoff val: ' + str(best_chop_cutoff_val))
print('Best chop val: ' + str(best_chop_num))
print('Best lookback: ' + str(best_lookback))
print('Best use adx val: ' + str(best_use_adx_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 224.31944499999958
Best risk/reward ratio: 0.5
Best spread: 0.025
Best chop cutoff val: 0.5
Best chop val: 14
-----------------------
Top results:
{'reward': 69, 'ratio': 1.0, 'spread_val': 0.05, 'chop_cutoff': 0.5, 'chop_num': 14}
{'reward': 224, 'ratio': 0.5, 'spread_val': 0.025, 'chop_cutoff': 0.5, 'chop_num': 14}
{'reward': 45, 'ratio': 1.0, 'spread_val': 0.025, 'chop_cutoff': 0.5, 'chop_num': 14}
{'reward': 94, 'ratio': 1.0, 'spread_val': 0.05, 'chop_cutoff': 0.618, 'chop_num': 14}
{'reward': 128, 'ratio': 1.0, 'spread_val': 0.05, 'chop_cutoff': 0.559, 'chop_num': 14}
{'reward': 93, 'ratio': 1.25, 'spread_val': 0.1, 'chop_cutoff': 0.618, 'chop_num': 14}
{'reward': 46, 'ratio': 0.5, 'spread_val': 0.025, 'chop_cutoff': 0.618, 'chop_num': 14}
{'reward': 109, 'ratio': 0.5, 'spread_val': 0.025, 'chop_cutoff': 0.559, 'chop_num': 14}
{'reward': 93, 'ratio': 0.75, 'spread_val': 0.05, 'chop_cutoff': 0.559, 'chop_num': 14}
{'reward': 172,

In [13]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best chop cutoff val: ' + str(best_chop_cutoff_val))
print('Best chop val: ' + str(best_chop_num))
print('Best lookback: ' + str(best_lookback))
print('Best use adx val: ' + str(best_use_adx_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 251.7917549999794
Best risk/reward ratio: 0.5
Best spread: 0.1
Best chop cutoff val: 0.618
Best chop val: 14
Best lookback: 1
-----------------------
Top results:
{'reward': 0, 'ratio': 1.5, 'spread_val': 0.025, 'chop_cutoff': 0.618, 'chop_num': 14}
{'reward': 0, 'ratio': 1.0, 'spread_val': 0.01, 'chop_cutoff': 0.5, 'chop_num': 14}
{'reward': 0, 'ratio': 1.5, 'spread_val': 0.01, 'chop_cutoff': 0.5, 'chop_num': 14}
{'reward': 0, 'ratio': 0.5, 'spread_val': 0.01, 'chop_cutoff': 0.618, 'chop_num': 14}
{'reward': 0, 'ratio': 1.0, 'spread_val': 0.025, 'chop_cutoff': 0.618, 'chop_num': 14}
{'reward': 0, 'ratio': 1.0, 'spread_val': 0.05, 'chop_cutoff': 0.618, 'chop_num': 14}
{'reward': 0, 'ratio': 1.5, 'spread_val': 0.01, 'chop_cutoff': 0.559, 'chop_num': 14}
{'reward': 0, 'ratio': 1.5, 'spread_val': 0.025, 'chop_cutoff': 0.559, 'chop_num': 14}
{'reward': 0, 'ratio': 1.0, 'spread_val': 0.025, 'chop_cutoff': 0.618, 'chop_num': 14}
{'reward':

In [11]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best chop cutoff val: ' + str(best_chop_cutoff_val))
print('Best chop val: ' + str(best_chop_num))
print('Best lookback: ' + str(best_lookback))
print('Best use adx val: ' + str(best_use_adx_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 138.4055599999995
Best risk/reward ratio: 1.5
Best spread: 0.1
Best chop cutoff val: 0.618
Best chop val: 14
Best lookback: 6
Best use adx val: False
-----------------------
Top results:
{'reward': -715, 'ratio': 1.0, 'spread_val': 0.1, 'chop_cutoff': 0.618, 'chop_num': 14, 'lookback': 6, 'use_adx': False}
{'reward': -896, 'ratio': 1.5, 'spread_val': 0.1, 'chop_cutoff': 0.559, 'chop_num': 14, 'lookback': 3, 'use_adx': True}
{'reward': -288, 'ratio': 1.5, 'spread_val': 0.1, 'chop_cutoff': 0.618, 'chop_num': 14, 'lookback': 1, 'use_adx': False}
{'reward': -758, 'ratio': 1.5, 'spread_val': 0.1, 'chop_cutoff': 0.559, 'chop_num': 14, 'lookback': 3, 'use_adx': False}
{'reward': -1021, 'ratio': 1.5, 'spread_val': 0.1, 'chop_cutoff': 0.618, 'chop_num': 14, 'lookback': 5, 'use_adx': True}
{'reward': -364, 'ratio': 1.5, 'spread_val': 0.1, 'chop_cutoff': 0.559, 'chop_num': 14, 'lookback': 5, 'use_adx': False}
{'reward': -684, 'ratio': 1.5, 'spr

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best chop cutoff val: ' + str(best_chop_cutoff_val))
print('Best chop val: ' + str(best_chop_num))
print('Best lookback: ' + str(best_lookback))
print('Best use adx val: ' + str(best_use_adx_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)